# SET UP

In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from osgeo import ogr, gdal, osr


# change working directory to project folder
import os
os.chdir('/Users/lacm/Documents/Stanford/*Capstone Research/Data')

# FINDING 3RD GRADE CATCHMENT ZONES

Using the CCD records, cross referenced with SABS data, to find a filtered list of all the attendance zones/schools that should be in the SABS/SABINS file for each year. Doing so requires finding a list of all the schools that serve 3rd grade for that school year. The final list of schools will be used to filter that year of SABINS data to only include those schools/attendance zones.

The SABINS files are already filtered by grade level so that they already only contain catchment zones for 3rd graders. As such, the years that use SABINS data (SY 2009-10 and SY 2011-12) will not need to be filtered.

<b>CCD LONGITUDINAL RECORD OF SCHOOLS</b>

In [4]:
# import ccd data (longitudinal)
ccdschools = pd.read_csv('/Users/lacm/Documents/Stanford/*Capstone Research/Data/clean school by year imputed data combined collapsed.csv', header=0, low_memory=False) 

In [8]:
# write leaid -> geoleaid crosswalk file to directory
# ccdschools_unclean.to_csv('clean school by year imputed data combined collapsed.csv')

In [5]:
# CLEAN CCD DATA

# convert variables to correct type
ccdschools['year'] = ccdschools['year'].astype('str').str[:4]
ccdschools['leaid'] = ccdschools['leaid'].astype('str')
ccdschools['geolea'] = ccdschools['geolea'].astype('str')
ccdschools['leaid'] = ccdschools['leaid'].astype('str')
ccdschools['ncessch'] = ccdschools['ncessch'].astype('str')
ccdschools['nces7_final'] = ccdschools['nces7_final'].astype('str')
ccdschools['charter'] = ccdschools['charter'].astype('int')
ccdschools['virtual'] = ccdschools['virtual'].astype('float')
ccdschools['magnet'] = ccdschools['magnet'].astype('int')
ccdschools['type'] = ccdschools['type'].astype('int')
ccdschools['grdlo'] = ccdschools['grdlo'].astype('float')
ccdschools['grdhi'] = ccdschools['grdhi'].astype('float')

# strip string columns
for column in ccdschools.columns:
    if ccdschools.dtypes[column] == 'object':
        ccdschools[column] = ccdschools[column].str.strip()

# add leading 0 to geolea, leaid, and school ids where it was chopped off from incorrect variable type
ccdschools.loc[ccdschools['geolea'].str.len() == 6, 'geolea'] = '0' + ccdschools['geolea']
ccdschools.loc[ccdschools['leaid'].str.len() == 6, 'leaid'] = '0' + ccdschools['leaid']
ccdschools.loc[ccdschools['ncessch'].str.len() == 11, 'ncessch'] = '0' + ccdschools['ncessch']
ccdschools.loc[ccdschools['nces7_final'].str.len() == 6, 'nces7_final'] = '0' + ccdschools['nces7_final']

# filter for only years with SABS/SABINS data; rename years so that year corresponds to second year of school year
ccdschools = ccdschools[(ccdschools.year == '1999') | (ccdschools.year == '2009') | (ccdschools.year == '2010') | 
                        (ccdschools.year == '2011') | (ccdschools.year == '2013') | (ccdschools.year == '2015')]

ccdschools['year'] = ccdschools['year'].replace(['1999', '2009', '2010', '2011', '2013', '2015'], 
                                                ['2000', '2010', '2011', '2012', '2014', '2016'])

# rename geolea column
ccdschools = ccdschools.rename(columns={'geolea' : 'geoleaid'})

# keep only relevant columns
ccdschools = ccdschools[['geoleaid','leaid', 'ncessch','nces7_final', 'year', 'state_name', 'county_name',  'grdlo', 
                         'grdhi', 'charter', 'magnet', 'virtual', 'type']]

# filter for only schools that serve 3rd grade
ccdschools = ccdschools[(ccdschools.grdlo <= 3) & (ccdschools.grdhi >= 3)]

# filter out virtual schools
ccdschools = ccdschools[ccdschools.virtual != 1]

# filter out magnet schools (only want schools with true catchment zones)
ccdschools = ccdschools[ccdschools.magnet == 0]

# keep only schools of type regular
# school types: 1 = regular, 2 = special education, 3 = vocational, 4 = alternative/other, 5 = ?
ccdschools = ccdschools[ccdschools.type == 1]

In [6]:
# creating separate CCD file for each year after full ccd dataset is clean

ccdschools_2000 = ccdschools[ccdschools.year == '2000']
ccdschools_2010 = ccdschools[ccdschools.year == '2010']
ccdschools_2011 = ccdschools[ccdschools.year == '2011']
ccdschools_2012 = ccdschools[ccdschools.year == '2012']
ccdschools_2014 = ccdschools[ccdschools.year == '2014']
ccdschools_2016 = ccdschools[ccdschools.year == '2016']

In [81]:
# creating separate CCD file for New York 2000
ccdschools_2000_ny = ccdschools_2000[(pd.isna(ccdschools_2000.state_name)) | (ccdschools_2000.state_name == "New York")]
ccdschools_2000_ny = ccdschools_2000_ny[3323:5646]
ccdschools_2000_ny['ccd_id'] = ccdschools_2000_ny['ncessch'].str[7:]

In [6]:
# download ccd files for each year
# ccdschools_2000.to_csv('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/CCD Schools 1999-2000.csv')
# ccdschools_2000_ny.to_csv('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/CCD Schools 1999-2000 - New York.csv')
# ccdschools_2010.to_csv('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2009-10/CCD Schools 2009-2010.csv')
# ccdschools_2011.to_csv('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2010-11/CCD Schools 2010-2011.csv')
# ccdschools_2012.to_csv('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2011-12/CCD Schools 2011-2012.csv')
# ccdschools_2014.to_csv('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2013-14/CCD Schools 2013-2014.csv')
# ccdschools_2016.to_csv('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2015-16/CCD Schools 2015-2016.csv')

<b>SABS/SAL SAPORITO DATASETS</b>

<b>SY 1999-2000</b>

In [82]:
# import excel file of all schools in the 2010-11 data year
sabs2000_baltimore = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Baltimore_City_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_baltimorecounty = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Baltimore_County_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_broward = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Broward_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_chicago = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Chicago_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_clark = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Clark_County_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_dallas = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Dallas_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_detroit = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Detroit_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_duval = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Duval_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_fairfax = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Fairfax_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_hillsborough = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Hillsborough_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_houston = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Houston_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_la = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_LA_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_miamidade = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Miami_Dade_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_milwaukee = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Milwaukee_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_montgomery = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Montgomery_Cnty_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_nyc = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_NYC_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_orange = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Orange_Cnty_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_palmbeach = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Palm_Beach_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_philadelphia = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Philadelphia_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_pinellas = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Pinellas_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_princegeorge = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_Prince_George_Cnty_fixed_geometries_reprojected.xlsx', header=0)
sabs2000_sandiego = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/elem_San_Diego_fixed_geometries_reprojected.xlsx', header=0)

In [83]:
# make column names lower
sabs2000_baltimore = sabs2000_baltimore.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_baltimore['ccd_id'] = sabs2000_baltimore['ccd_id'].astype(str)
sabs2000_baltimore.loc[sabs2000_baltimore['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_baltimore['ccd_id']
sabs2000_baltimore.loc[sabs2000_baltimore['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_baltimore['ccd_id']

# strip string columns
for column in sabs2000_baltimore.columns:
    if sabs2000_baltimore.dtypes[column] == 'object':
        sabs2000_baltimore[column] = sabs2000_baltimore[column].str.strip()
        
# add geoleaid for baltimore city schools
sabs2000_baltimore['geoleaid'] = '2400090'
sabs2000_baltimore['ncessch'] = sabs2000_baltimore['geoleaid'] + sabs2000_baltimore['ccd_id']

# merge baltimore city sabs dataset with ccd 2000 record of schools to find baltimore city schools that serve 3rd grade
# sabs2000_baltimore = sabs2000_baltimore.merge(ccdschools_2000, on='ncessch', how='left')
# sabs2000_baltimore = sabs2000_baltimore[(sabs2000_baltimore['grdlo'].notnull()) | (sabs2000_baltimore['grdhi'].notnull())]

In [84]:
# make column names lower
sabs2000_baltimorecounty = sabs2000_baltimorecounty.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_baltimorecounty['ccd_id'] = sabs2000_baltimorecounty['ccd_id'].astype(str)
sabs2000_baltimorecounty.loc[sabs2000_baltimorecounty['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_baltimorecounty['ccd_id']
sabs2000_baltimorecounty.loc[sabs2000_baltimorecounty['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_baltimorecounty['ccd_id']

# strip string columns
for column in sabs2000_baltimorecounty.columns:
    if sabs2000_baltimorecounty.dtypes[column] == 'object':
        sabs2000_baltimorecounty[column] = sabs2000_baltimorecounty[column].str.strip()

# add geoleaid for baltimore county schools
sabs2000_baltimorecounty['geoleaid'] = '2400120'
sabs2000_baltimorecounty['ncessch'] = sabs2000_baltimorecounty['geoleaid'] + sabs2000_baltimorecounty['ccd_id']
sabs2000_baltimorecounty


# merge baltimore county sabs dataset with ccd 2000 record of schools to find baltimore county schools that serve 3rd grade
sabs2000_baltimorecounty = sabs2000_baltimorecounty.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_baltimorecounty = sabs2000_baltimorecounty[(sabs2000_baltimorecounty['grdlo'].notnull()) | (sabs2000_baltimorecounty['grdhi'].notnull())]

In [85]:
# make column names lower
sabs2000_chicago = sabs2000_chicago.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_chicago['ccd_id'] = sabs2000_chicago['ccd_id'].astype(str)
sabs2000_chicago['ccd_id'] = sabs2000_chicago['ccd_id'].str.rstrip('.0')

sabs2000_chicago.loc[sabs2000_chicago['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_chicago['ccd_id']
sabs2000_chicago.loc[sabs2000_chicago['ccd_id'].str.len() == 1, 'ccd_id'] = '0000' + sabs2000_chicago['ccd_id']
sabs2000_chicago.loc[sabs2000_chicago['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_chicago['ccd_id']
sabs2000_chicago.loc[sabs2000_chicago['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_chicago['ccd_id']

# strip string columns
for column in sabs2000_chicago.columns:
    if sabs2000_chicago.dtypes[column] == 'object':
        sabs2000_chicago[column] = sabs2000_chicago[column].str.strip()

# add geoleaid for broward schools
sabs2000_chicago['geoleaid'] = '1709930'
sabs2000_chicago['ncessch'] = sabs2000_chicago['geoleaid'] + sabs2000_chicago['ccd_id']

# merge broward sabs dataset with ccd 2000 record of schools to find broward schools that serve 3rd grade
sabs2000_chicago = sabs2000_chicago.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_chicago = sabs2000_chicago[(sabs2000_chicago['grdlo'].notnull()) | (sabs2000_chicago['grdhi'].notnull())]

In [86]:
# make column names lower
sabs2000_broward = sabs2000_broward.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_broward['ccd_id'] = sabs2000_broward['ccd_id'].astype(str)
sabs2000_broward.loc[sabs2000_broward['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_broward['ccd_id']
sabs2000_broward.loc[sabs2000_broward['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_broward['ccd_id']

# strip string columns
for column in sabs2000_broward.columns:
    if sabs2000_broward.dtypes[column] == 'object':
        sabs2000_broward[column] = sabs2000_broward[column].str.strip()

# add geoleaid for broward schools
sabs2000_broward['geoleaid'] = '1200180'
sabs2000_broward['ncessch'] = sabs2000_broward['geoleaid'] + sabs2000_broward['ccd_id']

# merge broward sabs dataset with ccd 2000 record of schools to find broward schools that serve 3rd grade
sabs2000_broward = sabs2000_broward.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_broward = sabs2000_broward[(sabs2000_broward['grdlo'].notnull()) | (sabs2000_broward['grdhi'].notnull())]

In [87]:
# make column names lower
sabs2000_clark = sabs2000_clark.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_clark['ccd_id'] = sabs2000_clark['ccd_id'].astype(str)
sabs2000_clark.loc[sabs2000_clark['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_clark['ccd_id']
sabs2000_clark.loc[sabs2000_clark['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_clark['ccd_id']
sabs2000_clark.loc[sabs2000_clark['ccd_id'].str.len() == 1, 'ccd_id'] = '0000' + sabs2000_clark['ccd_id']

# strip string columns
for column in sabs2000_clark.columns:
    if sabs2000_clark.dtypes[column] == 'object':
        sabs2000_clark[column] = sabs2000_clark[column].str.strip()

# add geoleaid for clark schools
sabs2000_clark['geoleaid'] = '3200060'
sabs2000_clark['ncessch'] = sabs2000_clark['geoleaid'] + sabs2000_clark['ccd_id'].astype(str)

# merge clark sabs dataset with ccd 2000 record of schools to find clark county schools that serve 3rd grade
sabs2000_clark = sabs2000_clark.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_clark = sabs2000_clark[(sabs2000_clark['grdlo'].notnull()) | (sabs2000_clark['grdhi'].notnull())]

In [88]:
# make column names lower
sabs2000_dallas = sabs2000_dallas.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_dallas['ccd_id'] = sabs2000_dallas['ccd_id'].astype(str)
sabs2000_dallas['ccd_id'] = sabs2000_dallas['ccd_id'].str.rstrip('.0')

sabs2000_dallas.loc[sabs2000_dallas['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_dallas['ccd_id']
sabs2000_dallas.loc[sabs2000_dallas['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_dallas['ccd_id']
sabs2000_dallas.loc[sabs2000_dallas['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_dallas['ccd_id']
sabs2000_dallas['ccd_id'] = sabs2000_dallas['ccd_id'].replace(['00nan'], [''])

# strip string columns
for column in sabs2000_dallas.columns:
    if sabs2000_dallas.dtypes[column] == 'object':
        sabs2000_dallas[column] = sabs2000_dallas[column].str.strip()

# add geoleaid for dallas schools
sabs2000_dallas['geoleaid'] = '4816230'
sabs2000_dallas['ncessch'] = sabs2000_dallas['geoleaid'] + sabs2000_dallas['ccd_id'].astype(str)

# merge dallas sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_dallas = sabs2000_dallas.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_dallas = sabs2000_dallas[(sabs2000_dallas['grdlo'].notnull()) | (sabs2000_dallas['grdhi'].notnull())]

In [89]:
# make column names lower
sabs2000_detroit = sabs2000_detroit.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_detroit['ccd_id'] = sabs2000_detroit['ccd_id'].astype(str)
sabs2000_detroit['ccd_id'] = sabs2000_detroit['ccd_id'].str.rstrip('.0')

sabs2000_detroit.loc[sabs2000_detroit['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_detroit['ccd_id']
sabs2000_detroit.loc[sabs2000_detroit['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_detroit['ccd_id']
sabs2000_detroit.loc[sabs2000_detroit['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_detroit['ccd_id']

# strip string columns
for column in sabs2000_detroit.columns:
    if sabs2000_detroit.dtypes[column] == 'object':
        sabs2000_detroit[column] = sabs2000_detroit[column].str.strip()

# add geoleaid for detroit schools
sabs2000_detroit['geoleaid'] = '2601103'
sabs2000_detroit['ncessch'] = sabs2000_detroit['geoleaid'] + sabs2000_detroit['ccd_id'].astype(str)

# merge detroit sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_detroit = sabs2000_detroit.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_detroit = sabs2000_detroit[(sabs2000_detroit['grdlo'].notnull()) | (sabs2000_detroit['grdhi'].notnull())]

In [90]:
# make column names lower
sabs2000_duval = sabs2000_duval.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_duval['ccd_id'] = sabs2000_duval['ccd_id'].astype(str)
sabs2000_duval.loc[sabs2000_duval['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_duval['ccd_id']
sabs2000_duval.loc[sabs2000_duval['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_duval['ccd_id']

# strip string columns
for column in sabs2000_duval.columns:
    if sabs2000_duval.dtypes[column] == 'object':
        sabs2000_duval[column] = sabs2000_duval[column].str.strip()

# add geoleaid for baltimore city schools
sabs2000_duval['geoleaid'] = '1200480'
sabs2000_duval['ncessch'] = sabs2000_duval['geoleaid'] + sabs2000_duval['ccd_id'].astype(str)

# merge duval sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_duval = sabs2000_duval.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_duval = sabs2000_duval[(sabs2000_duval['grdlo'].notnull()) | (sabs2000_duval['grdhi'].notnull())]

In [91]:
# make column names lower
sabs2000_fairfax = sabs2000_fairfax.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_fairfax['ccd_id'] = sabs2000_fairfax['ccd_id'].astype(str)
sabs2000_fairfax.loc[sabs2000_fairfax['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_fairfax['ccd_id']
sabs2000_fairfax.loc[sabs2000_fairfax['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_fairfax['ccd_id']
sabs2000_fairfax.loc[sabs2000_fairfax['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_fairfax['ccd_id']

# strip string columns
for column in sabs2000_fairfax.columns:
    if sabs2000_fairfax.dtypes[column] == 'object':
        sabs2000_fairfax[column] = sabs2000_fairfax[column].str.strip()

# add geoleaid for fairfax schools
sabs2000_fairfax['geoleaid'] = '5101260'
sabs2000_fairfax['ncessch'] = sabs2000_fairfax['geoleaid'] + sabs2000_fairfax['ccd_id'].astype(str)

# merge fairfax sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_fairfax = sabs2000_fairfax.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_fairfax = sabs2000_fairfax[(sabs2000_fairfax['grdlo'].notnull()) | (sabs2000_fairfax['grdhi'].notnull())]

In [92]:
# make column names lower
sabs2000_hillsborough = sabs2000_hillsborough.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_hillsborough['ccd_id'] = sabs2000_hillsborough['ccd_id'].astype(str)
sabs2000_hillsborough.loc[sabs2000_hillsborough['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_hillsborough['ccd_id']
sabs2000_hillsborough.loc[sabs2000_hillsborough['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_hillsborough['ccd_id']

# strip string columns
for column in sabs2000_hillsborough.columns:
    if sabs2000_hillsborough.dtypes[column] == 'object':
        sabs2000_hillsborough[column] = sabs2000_hillsborough[column].str.strip()

# add geoleaid for hillsborough schools
sabs2000_hillsborough['geoleaid'] = '1200870'
sabs2000_hillsborough['ncessch'] = sabs2000_hillsborough['geoleaid'] + sabs2000_hillsborough['ccd_id'].astype(str)

# merge hillsborough sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_hillsborough = sabs2000_hillsborough.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_hillsborough = sabs2000_hillsborough[(sabs2000_hillsborough['grdlo'].notnull()) | (sabs2000_hillsborough['grdhi'].notnull())]

In [93]:
# make column names lower
sabs2000_houston = sabs2000_houston.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_houston['ccd_id'] = sabs2000_houston['ccd_id'].astype(str)
sabs2000_houston['ccd_id'] = sabs2000_houston['ccd_id'].str.rstrip('.0')

sabs2000_houston.loc[sabs2000_houston['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_houston['ccd_id']
sabs2000_houston.loc[sabs2000_houston['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_houston['ccd_id']
sabs2000_houston.loc[sabs2000_houston['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_houston['ccd_id']
sabs2000_houston['ccd_id'] = sabs2000_houston['ccd_id'].replace(['00nan'], [''])


# strip string columns
for column in sabs2000_houston.columns:
    if sabs2000_houston.dtypes[column] == 'object':
        sabs2000_houston[column] = sabs2000_houston[column].str.strip()

# add geoleaid for houston schools
sabs2000_houston['geoleaid'] = '4823640'
sabs2000_houston['ncessch'] = sabs2000_houston['geoleaid'] + sabs2000_houston['ccd_id'].astype(str)

# merge houston sabs dataset with ccd 2000 record of schools to find clark that serve 3rd grade
sabs2000_houston = sabs2000_houston.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_houston = sabs2000_houston[(sabs2000_houston['grdlo'].notnull()) | (sabs2000_houston['grdhi'].notnull())]

In [94]:
# make column names lower
sabs2000_la = sabs2000_la.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_la['ccd_id'] = sabs2000_la['ccd_id'].astype(str)
sabs2000_la['ccd_id'] = sabs2000_la['ccd_id'].str.rstrip('.0')

sabs2000_la.loc[sabs2000_la['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_la['ccd_id']
sabs2000_la.loc[sabs2000_la['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_la['ccd_id']
sabs2000_la.loc[sabs2000_la['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_la['ccd_id']
sabs2000_la.loc[sabs2000_la['ccd_id'].str.len() == 1, 'ccd_id'] = '0000' + sabs2000_la['ccd_id']
sabs2000_la['ccd_id'] = sabs2000_la['ccd_id'].replace(['00nan'], [''])

# strip string columns
for column in sabs2000_la.columns:
    if sabs2000_la.dtypes[column] == 'object':
        sabs2000_la[column] = sabs2000_la[column].str.strip()

# add geoleaid for los angeles schools
sabs2000_la['geoleaid'] = '0622710'
sabs2000_la['ncessch'] = sabs2000_la['geoleaid'] + sabs2000_la['ccd_id'].astype(str)

# merge los angeles sabs dataset with ccd 2000 record of schools to find clark that serve 3rd grade
sabs2000_la = sabs2000_la.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_la = sabs2000_la[(sabs2000_la['grdlo'].notnull()) | (sabs2000_la['grdhi'].notnull())]

In [95]:
# make column names lower
sabs2000_miamidade = sabs2000_miamidade.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_miamidade['ccd_id'] = sabs2000_miamidade['ccd_id'].astype(str)
sabs2000_miamidade['ccd_id'] = sabs2000_miamidade['ccd_id'].str.rstrip('.0')

sabs2000_miamidade.loc[sabs2000_miamidade['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_miamidade['ccd_id']
sabs2000_miamidade.loc[sabs2000_miamidade['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_miamidade['ccd_id']
sabs2000_miamidade.loc[sabs2000_miamidade['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_miamidade['ccd_id']
sabs2000_miamidade.loc[sabs2000_miamidade['ccd_id'].str.len() == 1, 'ccd_id'] = '0000' + sabs2000_miamidade['ccd_id']
sabs2000_miamidade['ccd_id'] = sabs2000_miamidade['ccd_id'].replace(['00nan'], [''])

# strip string columns
for column in sabs2000_miamidade.columns:
    if sabs2000_miamidade.dtypes[column] == 'object':
        sabs2000_miamidade[column] = sabs2000_miamidade[column].str.strip()

# add geoleaid for miami schools
sabs2000_miamidade['geoleaid'] = '1200390'
sabs2000_miamidade['ncessch'] = sabs2000_miamidade['geoleaid'] + sabs2000_miamidade['ccd_id'].astype(str)

# merge miami sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_miamidade = sabs2000_miamidade.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_miamidade = sabs2000_miamidade[(sabs2000_miamidade['grdlo'].notnull()) | (sabs2000_miamidade['grdhi'].notnull())]

In [96]:
# make column names lower
sabs2000_milwaukee = sabs2000_milwaukee.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_milwaukee['ccd_id'] = sabs2000_milwaukee['ccd_id'].astype(str)
sabs2000_milwaukee['ccd_id'] = sabs2000_milwaukee['ccd_id'].str.rstrip('.0')

sabs2000_milwaukee.loc[sabs2000_milwaukee['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_milwaukee['ccd_id']
sabs2000_milwaukee.loc[sabs2000_milwaukee['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_milwaukee['ccd_id']
sabs2000_milwaukee.loc[sabs2000_milwaukee['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_milwaukee['ccd_id']
sabs2000_milwaukee['ccd_id'] = sabs2000_milwaukee['ccd_id'].replace(['00nan'], [''])

# strip string columns
for column in sabs2000_milwaukee.columns:
    if sabs2000_milwaukee.dtypes[column] == 'object':
        sabs2000_milwaukee[column] = sabs2000_milwaukee[column].str.strip()

# add geoleaid for milwaukee schools
sabs2000_milwaukee['geoleaid'] = '5509600'
sabs2000_milwaukee['ncessch'] = sabs2000_milwaukee['geoleaid'] + sabs2000_milwaukee['ccd_id'].astype(str)

# merge milwaukee sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_milwaukee = sabs2000_milwaukee.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_milwaukee = sabs2000_milwaukee[(sabs2000_milwaukee['grdlo'].notnull()) | (sabs2000_milwaukee['grdhi'].notnull())]

In [97]:
# make column names lower
sabs2000_montgomery = sabs2000_montgomery.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_montgomery['ccd_id'] = sabs2000_montgomery['ccd_id'].astype(str)
sabs2000_montgomery['ccd_id'] = sabs2000_montgomery['ccd_id'].str.rstrip('.0')

sabs2000_montgomery.loc[sabs2000_montgomery['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_montgomery['ccd_id']
sabs2000_montgomery.loc[sabs2000_montgomery['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_montgomery['ccd_id']
sabs2000_montgomery.loc[sabs2000_montgomery['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_montgomery['ccd_id']
sabs2000_montgomery.loc[sabs2000_montgomery['ccd_id'].str.len() == 1, 'ccd_id'] = '0000' + sabs2000_montgomery['ccd_id']
sabs2000_montgomery['ccd_id'] = sabs2000_montgomery['ccd_id'].replace(['00nan'], [''])

# strip string columns
for column in sabs2000_montgomery.columns:
    if sabs2000_montgomery.dtypes[column] == 'object':
        sabs2000_montgomery[column] = sabs2000_montgomery[column].str.strip()

# add geoleaid for montgomery county schools
sabs2000_montgomery['geoleaid'] = '2400480'
sabs2000_montgomery['ncessch'] = sabs2000_montgomery['geoleaid'] + sabs2000_montgomery['ccd_id'].astype(str)

# merge montgomery sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_montgomery = sabs2000_montgomery.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_montgomery = sabs2000_montgomery[(sabs2000_montgomery['grdlo'].notnull()) | (sabs2000_montgomery['grdhi'].notnull())]

In [98]:
# make column names lower
sabs2000_nyc = sabs2000_nyc.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_nyc['ccd_id'] = sabs2000_nyc['ccd_id'].astype(str)
sabs2000_nyc['ccd_id'] = sabs2000_nyc['ccd_id'].str.rstrip('.0')

sabs2000_nyc.loc[sabs2000_nyc['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_nyc['ccd_id']
sabs2000_nyc.loc[sabs2000_nyc['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_nyc['ccd_id']
sabs2000_nyc.loc[sabs2000_nyc['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_nyc['ccd_id']
sabs2000_nyc['ccd_id'] = sabs2000_nyc['ccd_id'].replace(['00nan'], [''])

# strip string columns
for column in sabs2000_nyc.columns:
    if sabs2000_nyc.dtypes[column] == 'object':
        sabs2000_nyc[column] = sabs2000_nyc[column].str.strip()

# merge nyc sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_nyc = sabs2000_nyc.merge(ccdschools_2000_ny, on='ccd_id', how='left')
sabs2000_nyc = sabs2000_nyc[(sabs2000_nyc['grdlo'].notnull()) | (sabs2000_nyc['grdhi'].notnull())]

In [99]:
# make column names lower
sabs2000_orange = sabs2000_orange.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_orange['ccd_id'] = sabs2000_orange['ccd_id'].astype(str)
sabs2000_orange['ccd_id'] = sabs2000_orange['ccd_id'].str.rstrip('.0')

sabs2000_orange.loc[sabs2000_orange['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_orange['ccd_id']
sabs2000_orange.loc[sabs2000_orange['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_orange['ccd_id']
sabs2000_orange.loc[sabs2000_orange['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_orange['ccd_id']
sabs2000_orange['ccd_id'] = sabs2000_orange['ccd_id'].replace(['00nan'], [''])

# strip string columns
for column in sabs2000_orange.columns:
    if sabs2000_orange.dtypes[column] == 'object':
        sabs2000_orange[column] = sabs2000_orange[column].str.strip()

# add geoleaid for orange schools
sabs2000_orange['geoleaid'] = '1201440'
sabs2000_orange['ncessch'] = sabs2000_orange['geoleaid'] + sabs2000_orange['ccd_id'].astype(str)

# merge orange sabs dataset with ccd 2000 record of schools to find clark county schools that serve 3rd grade
sabs2000_orange = sabs2000_orange.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_orange = sabs2000_orange[(sabs2000_orange['grdlo'].notnull()) | (sabs2000_orange['grdhi'].notnull())]

In [100]:
# make column names lower
sabs2000_palmbeach = sabs2000_palmbeach.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_palmbeach['ccd_id'] = sabs2000_palmbeach['ccd_id'].astype(str)
sabs2000_palmbeach.loc[sabs2000_palmbeach['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_palmbeach['ccd_id']
sabs2000_palmbeach.loc[sabs2000_palmbeach['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_palmbeach['ccd_id']

# strip string columns
for column in sabs2000_palmbeach.columns:
    if sabs2000_palmbeach.dtypes[column] == 'object':
        sabs2000_palmbeach[column] = sabs2000_palmbeach[column].str.strip()

# add geoleaid for palm beach schools
sabs2000_palmbeach['geoleaid'] = '1201500'
sabs2000_palmbeach['ncessch'] = sabs2000_palmbeach['geoleaid'] + sabs2000_palmbeach['ccd_id'].astype(str)

# merge palm beach sabs dataset with ccd 2000 record of schools to find clark county schools that serve 3rd grade
sabs2000_palmbeach = sabs2000_palmbeach.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_palmbeach = sabs2000_palmbeach[(sabs2000_palmbeach['grdlo'].notnull()) | (sabs2000_palmbeach['grdhi'].notnull())]

In [101]:
# make column names lower
sabs2000_philadelphia = sabs2000_philadelphia.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_philadelphia['ccd_id'] = sabs2000_philadelphia['ccd_id'].astype(str)
sabs2000_philadelphia.loc[sabs2000_philadelphia['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_philadelphia['ccd_id']
sabs2000_philadelphia.loc[sabs2000_philadelphia['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_philadelphia['ccd_id']

# strip string columns
for column in sabs2000_philadelphia.columns:
    if sabs2000_philadelphia.dtypes[column] == 'object':
        sabs2000_philadelphia[column] = sabs2000_philadelphia[column].str.strip()

# add geoleaid for philly schools
sabs2000_philadelphia['geoleaid'] = '4218990'
sabs2000_philadelphia['ncessch'] = sabs2000_philadelphia['geoleaid'] + sabs2000_philadelphia['ccd_id'].astype(str)

# merge philly sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_philadelphia = sabs2000_philadelphia.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_philadelphia = sabs2000_philadelphia[(sabs2000_philadelphia['grdlo'].notnull()) | (sabs2000_philadelphia['grdhi'].notnull())]

In [102]:
# make column names lower
sabs2000_pinellas = sabs2000_pinellas.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_pinellas['ccd_id'] = sabs2000_pinellas['ccd_id'].astype(str)
sabs2000_pinellas['ccd_id'] = sabs2000_pinellas['ccd_id'].str.rstrip('.0')

sabs2000_pinellas.loc[sabs2000_pinellas['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_pinellas['ccd_id']
sabs2000_pinellas.loc[sabs2000_pinellas['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_pinellas['ccd_id']
sabs2000_pinellas.loc[sabs2000_pinellas['ccd_id'].str.len() == 2, 'ccd_id'] = '000' + sabs2000_pinellas['ccd_id']
sabs2000_pinellas['ccd_id'] = sabs2000_pinellas['ccd_id'].replace(['00nan'], [''])

# strip string columns
for column in sabs2000_pinellas.columns:
    if sabs2000_pinellas.dtypes[column] == 'object':
        sabs2000_pinellas[column] = sabs2000_pinellas[column].str.strip()

# add geoleaid for pinellas schools
sabs2000_pinellas['geoleaid'] = '1201560'
sabs2000_pinellas['ncessch'] = sabs2000_pinellas['geoleaid'] + sabs2000_pinellas['ccd_id'].astype(str)

# merge pinellas sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_pinellas = sabs2000_pinellas.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_pinellas = sabs2000_pinellas[(sabs2000_pinellas['grdlo'].notnull()) | (sabs2000_pinellas['grdhi'].notnull())]

In [103]:
# make column names lower
sabs2000_princegeorge = sabs2000_princegeorge.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_princegeorge['ccd_id'] = sabs2000_princegeorge['ccd_id'].astype(str)
sabs2000_princegeorge.loc[sabs2000_princegeorge['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_princegeorge['ccd_id']
sabs2000_princegeorge.loc[sabs2000_princegeorge['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_princegeorge['ccd_id']


# strip string columns
for column in sabs2000_princegeorge.columns:
    if sabs2000_princegeorge.dtypes[column] == 'object':
        sabs2000_princegeorge[column] = sabs2000_princegeorge[column].str.strip()

# add geoleaid for prince george's schools
sabs2000_princegeorge['geoleaid'] = '2400510'
sabs2000_princegeorge['ncessch'] = sabs2000_princegeorge['geoleaid'] + sabs2000_princegeorge['ccd_id'].astype(str)

# merge prince george's sabs dataset with ccd 2000 record of schools to find schools that serve 3rd grade
sabs2000_princegeorge = sabs2000_princegeorge.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_princegeorge = sabs2000_princegeorge[(sabs2000_princegeorge['grdlo'].notnull()) | (sabs2000_princegeorge['grdhi'].notnull())]

In [104]:
# make column names lower
sabs2000_sandiego = sabs2000_sandiego.rename(str.lower, axis='columns')

# fix stripped ccd_ids --> should be 6 characters, so add 0s where they are less
sabs2000_sandiego['ccd_id'] = sabs2000_sandiego['ccd_id'].astype(str)
sabs2000_sandiego['ccd_id'] = sabs2000_sandiego['ccd_id'].str.rstrip('.0')

sabs2000_sandiego.loc[sabs2000_sandiego['ccd_id'].str.len() == 4, 'ccd_id'] = '0' + sabs2000_sandiego['ccd_id']
sabs2000_sandiego.loc[sabs2000_sandiego['ccd_id'].str.len() == 3, 'ccd_id'] = '00' + sabs2000_sandiego['ccd_id']
sabs2000_sandiego['ccd_id'] = sabs2000_sandiego['ccd_id'].replace(['00nan'], [''])

# strip string columns
for column in sabs2000_sandiego.columns:
    if sabs2000_sandiego.dtypes[column] == 'object':
        sabs2000_sandiego[column] = sabs2000_sandiego[column].str.strip()

# add geoleaid for san diego schools
sabs2000_sandiego['geoleaid'] = '0634320'
sabs2000_sandiego['ncessch'] = sabs2000_sandiego['geoleaid'] + sabs2000_sandiego['ccd_id'].astype(str)

# merge san diego sabs dataset with ccd 2000 record of schools to find clark county schools that serve 3rd grade
sabs2000_sandiego = sabs2000_sandiego.merge(ccdschools_2000, on='ncessch', how='left')
sabs2000_sandiego = sabs2000_sandiego[(sabs2000_sandiego['grdlo'].notnull()) | (sabs2000_sandiego['grdhi'].notnull())]

In [105]:
# GET SINGLE STRING OF ALL SCHOOLS/CATCHMENT ZONES TO INCLUDE IN SHAPEFILE FOR 1999 - 2000

sabs_2000 = []

# add baltimore
for ccd_id in list(sabs2000_baltimore['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)
    
# add baltimore county
for ccd_id in list(sabs2000_baltimorecounty['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)
    
# add broward county
for ccd_id in list(sabs2000_broward['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)
    
# add chicago
for ccd_id in list(sabs2000_chicago['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)
    
# add clark county
for ccd_id in list(sabs2000_clark['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)
    
# add dallas
for ccd_id in list(sabs2000_dallas['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add detroit
for ccd_id in list(sabs2000_detroit['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add duval county
for ccd_id in list(sabs2000_duval['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add fairfax county
for ccd_id in list(sabs2000_fairfax['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add hillsborough county
for ccd_id in list(sabs2000_hillsborough['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add houston
for ccd_id in list(sabs2000_houston['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add los angeles
for ccd_id in list(sabs2000_la['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add miami dade
for ccd_id in list(sabs2000_miamidade['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)
    
# add milwaukee
for ccd_id in list(sabs2000_milwaukee['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add montgomery county
for ccd_id in list(sabs2000_montgomery['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add new york city
for ccd_id in list(sabs2000_nyc['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add orange county
for ccd_id in list(sabs2000_orange['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add palm beach 
for ccd_id in list(sabs2000_palmbeach['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add philadelphia 
for ccd_id in list(sabs2000_philadelphia['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add pinellas county 
for ccd_id in list(sabs2000_pinellas['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add prince george's county  
for ccd_id in list(sabs2000_princegeorge['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)

# add san diego 
for ccd_id in list(sabs2000_sandiego['ccd_id']):
    if ccd_id not in sabs_2000:
        sabs_2000.append(ccd_id)
    
filter_2000 = ''
for ccd_id in sabs_2000:
    filter_2000 += "CCD_ID = '" + ccd_id + "' OR "
filter_2000 = filter_2000[:-4]

In [106]:
filter_2000

"CCD_ID = '00319' OR CCD_ID = '00266' OR CCD_ID = '00169' OR CCD_ID = '00196' OR CCD_ID = '00264' OR CCD_ID = '00320' OR CCD_ID = '00258' OR CCD_ID = '00246' OR CCD_ID = '00220' OR CCD_ID = '00215' OR CCD_ID = '00332' OR CCD_ID = '00189' OR CCD_ID = '00180' OR CCD_ID = '00335' OR CCD_ID = '00278' OR CCD_ID = '00178' OR CCD_ID = '00153' OR CCD_ID = '00239' OR CCD_ID = '01153' OR CCD_ID = '00184' OR CCD_ID = '00281' OR CCD_ID = '00201' OR CCD_ID = '00234' OR CCD_ID = '00143' OR CCD_ID = '00329' OR CCD_ID = '00276' OR CCD_ID = '00152' OR CCD_ID = '00233' OR CCD_ID = '00149' OR CCD_ID = '00229' OR CCD_ID = '00256' OR CCD_ID = '00192' OR CCD_ID = '00307' OR CCD_ID = '00270' OR CCD_ID = '00285' OR CCD_ID = '00193' OR CCD_ID = '00312' OR CCD_ID = '00205' OR CCD_ID = '00602' OR CCD_ID = '00334' OR CCD_ID = '00323' OR CCD_ID = '00204' OR CCD_ID = '00337' OR CCD_ID = '00273' OR CCD_ID = '00305' OR CCD_ID = '00208' OR CCD_ID = '00179' OR CCD_ID = '00310' OR CCD_ID = '00195' OR CCD_ID = '00321' OR

<b>SY 2009-10</b>

In [60]:
# sabs2010_unclean = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2009-10/SABS:SABINS/Clean/SABINS/PY_SABINS_0910_03_US_fixed_geometries_reprojected.xlsx', header=0)

In [61]:
# sabs2010 = sabs2010_unclean

In [62]:
# sabs2010['SDUNIA_g'] = sabs2010['SDUNIA_g'].astype('str')

In [63]:
# sabs2010['SDUNIA_g'].str.len().unique()

In [64]:
# sabs2010.loc[sabs2010['SDUNIA_g'].str.len() == 6, 'SDUNIA_g'] = '0' + sabs2010['SDUNIA_g']

In [65]:
# sabs2010 = pd.merge(sabs2010, ccdschools_2010, how='left', left_on='SDUNIA_g', right_on='leaid')

In [66]:
# len(sabs2010['geoleaid'].unique())

<b>SY 2010-11</b>

In [ ]:
# import excel file of all schools in SABS 2010-11
sabs2011 = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2010-11/SABS:SABINS/Raw/SABS (NCES Website)/SAA1011.xlsx', header=0)

In [ ]:
# CLEAN SABS 2011 DATASET

# convert variable names to lowercase
sabs2011.columns = sabs2011.columns.str.lower()

# change variables to correct types
sabs2011['ncessch'] = sabs2011['ncessch'].astype('str')
sabs2011['leaid'] = sabs2011['leaid'].astype('str')

# add leading 0 to geolea, leaid, and school ids where it was chopped off from incorrect variable type
sabs2011.loc[sabs2011['ncessch'].str.len() == 11, 'ncessch'] = '0' + sabs2011['ncessch']
sabs2011.loc[sabs2011['leaid'].str.len() == 6, 'leaid'] = '0' + sabs2011['leaid']

# change lowest grade (gslo) and highest grade (gshi) to numeric values
sabs2011['gslo'] = sabs2011['gslo'].replace(['KG', 'PK', 'UG', 'N'], 
                                            ['0', '-1', np.nan, np.nan])

sabs2011['gshi'] = sabs2011['gshi'].replace(['KG', 'PK', 'UG', 'N'], 
                                            ['0', '-1', np.nan, np.nan])

# change gslo and gshi to numeric values
sabs2011['gslo'] = sabs2011['gslo'].astype('int')
sabs2011['gshi'] = sabs2011['gshi'].astype('int')

# keep only schools that serve 3rd grade
sabs2011 = sabs2011[(sabs2011.gslo <= 3) & (sabs2011.gshi >= 3)]

# merge with ccd school list to get info on schools that should be removed from SABS list
sabs2011 = pd.merge(sabs2011, ccdschools_2011_unclean, on='ncessch', how='left')

# filter out virtual schools
sabs2011 = sabs2011[sabs2011.virtual != 1]

# filter out magnet schools
sabs2011 = sabs2011[sabs2011.magnet != 1]

# filter out non-regular schools 
# school types: 1 = regular, 2 = special education, 3 = vocational, 4 = alternative/other, 5 = ?
sabs2011 = sabs2011[(sabs2011.type != 2) & (sabs2011.type != 3) & (sabs2011.type != 4) & (sabs2011.type != 5)] 

In [ ]:
# GET SINGLE STRING OF ALL SCHOOLS/CATCHMENT ZONES TO INCLUDE IN SHAPEFILE FOR 2013-14
filter_2011 = ''
for ncessch in list(sabs2011['ncessch']):
    filter_2011 += "ncessch = '" + ncessch + "' OR "
filter_2011 = filter_2011[:-4]

In [ ]:
filter_2011

<b>SY 2011-12</b>

In [72]:
# import excel file of all schools in SABS 2010-11
# sabs2012_unclean = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2011-12/SABINS/Clean/PY_SABINS_1112_03_US_fixed_geometries_reprojected.xlsx', header=0)

In [73]:
# sabs2012 = sabs2012_unclean

In [74]:
# sabs2012['SDUNIA_g'] = sabs2012['SDUNIA_g'].astype('str')

In [75]:
# sabs2012['SDUNIA_g'].str.len().unique()

In [76]:
# sabs2012 = pd.merge(sabs2012, ccdschools_2012, how='left', left_on='SDUNIA_g', right_on='leaid')

In [77]:
# len(sabs2012['geoleaid'].unique())

<b>SY 2013-14</b>

In [41]:
# import excel file of all schools in SABS 2013-14
sabs2014_unclean = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2013-14/SABS/Raw/SABS_1314.xlsx', header=0)

In [42]:
sabs2014 = sabs2014_unclean

In [43]:
# CLEAN SABS 2014 DATASET

# convert variable names to lowercase
sabs2014.columns = sabs2014.columns.str.lower()

# change variables to correct types
sabs2014['ncessch'] = sabs2014['ncessch'].astype('str')
sabs2014['leaid'] = sabs2014['leaid'].astype('str')

# add leading 0 to geolea, leaid, and school ids where it was chopped off from incorrect variable type
sabs2014.loc[sabs2014['ncessch'].str.len() == 11, 'ncessch'] = '0' + sabs2014['ncessch']
sabs2014.loc[sabs2014['leaid'].str.len() == 6, 'leaid'] = '0' + sabs2014['leaid']

# change lowest grade (gslo) and highest grade (gshi) to numeric values
sabs2014['gslo'] = sabs2014['gslo'].replace(['KG', 'PK', 'UG', 'N'], 
                                            ['0', '-1', np.nan, np.nan])

sabs2014['gshi'] = sabs2014['gshi'].replace(['KG', 'PK', 'UG', 'N'], 
                                            ['0', '-1', np.nan, np.nan])

# change gslo and gshi to numeric values
sabs2014['gslo'] = sabs2014['gslo'].astype('float')
sabs2014['gshi'] = sabs2014['gshi'].astype('float')

# keep only schools that serve 3rd grade
sabs2014 = sabs2014[(sabs2014.gslo <= 3) & (sabs2014.gshi >= 3)]

# merge with ccd school list to get info on schools that should be removed from SABS list
sabs2014 = pd.merge(sabs2014, ccdschools_2014, on='ncessch', how='left')

# remove non-response districts for SABS 2013-14 from 2014 schools list
nonresponse_2014 = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2013-14/SABS/Raw/District_Nonresponse.xls', header=0)
nonresponse_2014['LEA_ID'] = nonresponse_2014['LEA_ID'].astype('str')
nonresponse_2014.loc[nonresponse_2014['LEA_ID'].str.len() == 6, 'LEA_ID'] = '0' + nonresponse_2014['LEA_ID']

for leaid in list(nonresponse_2014['LEA_ID']):
    sabs2014 = sabs2014[sabs2014.geoleaid != leaid]

# filter out virtual schools
sabs2014 = sabs2014[sabs2014.virtual != 1]

# filter out magnet schools
sabs2014 = sabs2014[sabs2014.magnet != 1]

# filter out non-regular schools 
# school types: 1 = regular, 2 = special education, 3 = vocational, 4 = alternative/other, 5 = ?
sabs2014 = sabs2014[(sabs2014.type != 2) & (sabs2014.type != 3) & (sabs2014.type != 4) & (sabs2014.type != 5)] 

In [44]:
# GET SINGLE STRING OF ALL SCHOOLS/CATCHMENT ZONES TO INCLUDE IN SHAPEFILE FOR 2013-14
filter_2014 = ''
for ncessch in list(sabs2014['ncessch']):
    filter_2014 += "ncessch = '" + ncessch + "' OR "
filter_2014 = filter_2014[:-4]

In [15]:
filter_2014

"ncessch = '010000500889' OR ncessch = '010000600876' OR ncessch = '010000600877' OR ncessch = '010000600880' OR ncessch = '010000601685' OR ncessch = '010000602209' OR ncessch = '010000699991' OR ncessch = '010000700091' OR ncessch = '010000700248' OR ncessch = '010000700342' OR ncessch = '010000701483' OR ncessch = '010000701738' OR ncessch = '010000701739' OR ncessch = '010000701740' OR ncessch = '010000701742' OR ncessch = '010000701743' OR ncessch = '010000702091' OR ncessch = '010000800303' OR ncessch = '010000800839' OR ncessch = '010000800851' OR ncessch = '010000801423' OR ncessch = '010000801485' OR ncessch = '010000802097' OR ncessch = '010000802133' OR ncessch = '010000899991' OR ncessch = '010001102094' OR ncessch = '010001200884' OR ncessch = '010001301498' OR ncessch = '010003000005' OR ncessch = '010006001467' OR ncessch = '010009000015' OR ncessch = '010009000017' OR ncessch = '010009000020' OR ncessch = '010009000021' OR ncessch = '010009001782' OR ncessch = '01000909

<b>SY 2015-16</b>

In [45]:
# import excel file of all schools in SABS 2015-16
sabs2016_unclean = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2015-16/SABS/Raw/SABS_1516.xlsx', header=0)

In [46]:
sabs2016 = sabs2016_unclean

In [47]:
# CLEAN SABS 2016 DATASET

# convert variable names to lowercase
sabs2016.columns = sabs2016.columns.str.lower()

# change variables to correct types
sabs2016['ncessch'] = sabs2016['ncessch'].astype('str')
sabs2016['leaid'] = sabs2016['leaid'].astype('str')

# add leading 0 to geolea, leaid, and school ids where it was chopped off from incorrect variable type
sabs2016.loc[sabs2016['ncessch'].str.len() == 11, 'ncessch'] = '0' + sabs2016['ncessch']
sabs2016.loc[sabs2016['leaid'].str.len() == 6, 'leaid'] = '0' + sabs2016['leaid']

# change lowest grade (gslo) and highest grade (gshi) to numeric values
sabs2016['gslo'] = sabs2016['gslo'].replace(['KG', 'PK', 'UG', 'N'], 
                                            ['0', '-1', np.nan, np.nan])

sabs2016['gshi'] = sabs2016['gshi'].replace(['KG', 'PK', 'UG', 'N'], 
                                            ['0', '-1', np.nan, np.nan])

# change gslo and gshi to numeric values
sabs2016['gslo'] = sabs2016['gslo'].astype('float')
sabs2016['gshi'] = sabs2016['gshi'].astype('float')

# keep only schools that serve 3rd grade
sabs2016 = sabs2016[(sabs2016.gslo <= 3) & (sabs2016.gshi >= 3)]

# merge with ccd school list to get info on schools that should be removed from SABS list
sabs2016 = pd.merge(sabs2016, ccdschools_2016, on='ncessch', how='left')

# remove non-response districts for SABS 2015-16 from 2016 schools list
nonresponse_2016 = pd.read_excel('/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2015-16/SABS/Raw/District_Nonresponse.xls', header=0)
nonresponse_2016['Lea_Id'] = nonresponse_2016['Lea_Id'].astype('str')
nonresponse_2016.loc[nonresponse_2016['Lea_Id'].str.len() == 6, 'Lea_Id'] = '0' + nonresponse_2016['Lea_Id']

for leaid in list(nonresponse_2016['Lea_Id']):
    sabs2016 = sabs2016[sabs2016.geoleaid != leaid]


# filter out virtual schools
sabs2016 = sabs2016[sabs2016.virtual != 1]

# filter out magnet schools
sabs2016 = sabs2016[sabs2016.magnet != 1]

# filter out non-regular schools 
# school types: 1 = regular, 2 = special education, 3 = vocational, 4 = alternative/other, 5 = ?
sabs2016 = sabs2016[(sabs2016.type != 2) & (sabs2016.type != 3) & (sabs2016.type != 4) & (sabs2016.type != 5)] 


In [48]:
# GET SINGLE STRING OF ALL SCHOOLS/CATCHMENT ZONES TO INCLUDE IN SHAPEFILE FOR 2015-16
filter_2016 = ''
for ncessch in list(sabs2016['ncessch']):
    filter_2016 += "ncessch = '" + ncessch + "' OR "
filter_2016 = filter_2016[:-4]

In [20]:
filter_2016

"ncessch = '010000500889' OR ncessch = '010000700091' OR ncessch = '010000700248' OR ncessch = '010000700342' OR ncessch = '010000701483' OR ncessch = '010000701738' OR ncessch = '010000701739' OR ncessch = '010000701740' OR ncessch = '010000701742' OR ncessch = '010000701743' OR ncessch = '010000702091' OR ncessch = '010000799991' OR ncessch = '010000800303' OR ncessch = '010000800839' OR ncessch = '010000800851' OR ncessch = '010000801423' OR ncessch = '010000801485' OR ncessch = '010000802097' OR ncessch = '010000802133' OR ncessch = '010000899991' OR ncessch = '010001102094' OR ncessch = '010001200884' OR ncessch = '010001301498' OR ncessch = '010003000005' OR ncessch = '010006001467' OR ncessch = '010009000015' OR ncessch = '010009000017' OR ncessch = '010009000020' OR ncessch = '010009000021' OR ncessch = '010009001782' OR ncessch = '010009099991' OR ncessch = '010010000022' OR ncessch = '010012001471' OR ncessch = '010012001474' OR ncessch = '010012001475' OR ncessch = '01001200

# FILTER SABS VECTOR LAYERS

<b>SY 1999-2000</b>

In [132]:
inShapefile = '/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 1999-2000/Sal Saporito Data/Clean/SABS 1999-2000.shp'
inDriver = ogr.GetDriverByName("ESRI Shapefile")
inDataSource = inDriver.Open(inShapefile, 0)
inLayer = inDataSource.GetLayer()
# change filter for each year
inLayer.SetAttributeFilter(filter_2000)

0

In [124]:
# number of features after filter
inLayer.GetFeatureCount()

3799

In [133]:
# Create the output LayerS
outShapefile = os.path.join( os.path.split( inShapefile )[0], "3rd Grade Only SABS 1999-2000.shp" )
outDriver = ogr.GetDriverByName("ESRI Shapefile")

In [134]:
# Create the output shapefile
outDataSource = outDriver.CreateDataSource(outShapefile)
out_lyr_name = os.path.splitext( os.path.split( outShapefile )[1] )[0]
outLayer = outDataSource.CreateLayer( out_lyr_name, srs=inLayer.GetSpatialRef(),geom_type=ogr.wkbMultiPolygon )

In [135]:
# Add input Layer Fields to the output Layer if it is the one we want
inLayerDefn = inLayer.GetLayerDefn()
for i in range(0, inLayerDefn.GetFieldCount()):
    fieldDefn = inLayerDefn.GetFieldDefn(i)
    fieldName = fieldDefn.GetName()
#     if fieldName not in field_name_target:
#         continue
    outLayer.CreateField(fieldDefn)

In [136]:
# Get the output Layer's Feature Definition
outLayerDefn = outLayer.GetLayerDefn()

In [137]:
# Add features to the ouput Layer
for inFeature in inLayer:
    # Create output Feature
    outFeature = ogr.Feature(outLayerDefn)

    # Add field values from input Layer
    for i in range(0, outLayerDefn.GetFieldCount()):
        fieldDefn = outLayerDefn.GetFieldDefn(i)
        fieldName = fieldDefn.GetName()
#         if fieldName not in field_name_target:
#             continue

        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(),
            inFeature.GetField(i))
    
    # Set geometry as centroid
    geom = inFeature.GetGeometryRef()
    outFeature.SetGeometry(geom.Clone())
    # Add new feature to output Layer
    outLayer.CreateFeature(outFeature)
    outFeature = None

Warning 1: Value 131370867.095577151 of field Shape_Area of feature 146 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 122993391.008997589 of field Shape_Area of feature 151 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 103105608.306539714 of field Shape_Area of feature 152 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 110421687.484121338 of field Shape_Area of feature 153 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 608502912 of field AREA of feature 674 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 473059871.99999994 of field AREA of feature 675 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 407197600 of field AREA of feature 676 not suc

Warning 1: Value 2134396339.44000006 of field AREA of feature 1158 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 198205579.383923382 of field Shape_Area of feature 1158 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 2391675469.28999996 of field AREA of feature 1159 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 222242629.240078121 of field Shape_Area of feature 1159 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 239437017.268999994 of field AREA of feature 1161 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 266042107.689999998 of field AREA of feature 1169 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 135739332.979000002 of field AREA of feat

In [138]:
# Save and close DataSources
inDataSource = None
outDataSource = None

<b>SY 2010-11</b>

In [131]:
inShapefile = '/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2010-11/SABS:SABINS/Clean/SABS/SAA1011_Fixed_Geometries_Reprojected.shp'
inDriver = ogr.GetDriverByName("ESRI Shapefile")
inDataSource = inDriver.Open(inShapefile, 0)
inLayer = inDataSource.GetLayer()
inLayer.SetAttributeFilter(filter_2011)

NameError: name 'filter_2011' is not defined

In [22]:
# number of features after filter
inLayer.GetFeatureCount()

18834

In [10]:
# Create the output LayerS
outShapefile = os.path.join( os.path.split( inShapefile )[0], "3rd Grade Only SABS 2010-2011.shp" )
outDriver = ogr.GetDriverByName("ESRI Shapefile")

In [11]:
# Create the output shapefile
outDataSource = outDriver.CreateDataSource(outShapefile)
out_lyr_name = os.path.splitext( os.path.split( outShapefile )[1] )[0]
outLayer = outDataSource.CreateLayer( out_lyr_name, srs=inLayer.GetSpatialRef(),geom_type=ogr.wkbMultiPolygon )

In [12]:
# Add input Layer Fields to the output Layer if it is the one we want
inLayerDefn = inLayer.GetLayerDefn()
for i in range(0, inLayerDefn.GetFieldCount()):
    fieldDefn = inLayerDefn.GetFieldDefn(i)
    fieldName = fieldDefn.GetName()
#     if fieldName not in field_name_target:
#         continue
    outLayer.CreateField(fieldDefn)

In [13]:
# Get the output Layer's Feature Definition
outLayerDefn = outLayer.GetLayerDefn()

In [14]:
# Add features to the ouput Layer
for inFeature in inLayer:
    # Create output Feature
    outFeature = ogr.Feature(outLayerDefn)

    # Add field values from input Layer
    for i in range(0, outLayerDefn.GetFieldCount()):
        fieldDefn = outLayerDefn.GetFieldDefn(i)
        fieldName = fieldDefn.GetName()
#         if fieldName not in field_name_target:
#             continue

        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(),
            inFeature.GetField(i))
    
    # Set geometry as centroid
    geom = inFeature.GetGeometryRef()
    outFeature.SetGeometry(geom.Clone())
    # Add new feature to output Layer
    outLayer.CreateFeature(outFeature)
    outFeature = None

In [15]:
outLayer.GetFeatureCount()

18834

In [16]:
# Save and close DataSources
inDataSource = None
outDataSource = None

<b>SY 2013-14</b>

In [59]:
inShapefile = '/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2013-14/SABS/Clean/SABS_1314_Primary_Fixed_Geometries_Reprojected.shp'
inDriver = ogr.GetDriverByName("ESRI Shapefile")
inDataSource = inDriver.Open(inShapefile, 0)
inLayer = inDataSource.GetLayer()
# change filter for each year
inLayer.SetAttributeFilter(filter_2014)

0

In [30]:
# number of features after filter
inLayer.GetFeatureCount()

39092

In [60]:
# Create the output LayerS
outShapefile = os.path.join( os.path.split( inShapefile )[0], "3rd Grade Only SABS 2013-2014.shp" )
outDriver = ogr.GetDriverByName("ESRI Shapefile")

In [61]:
# Create the output shapefile
outDataSource = outDriver.CreateDataSource(outShapefile)
out_lyr_name = os.path.splitext( os.path.split( outShapefile )[1] )[0]
outLayer = outDataSource.CreateLayer( out_lyr_name, srs=inLayer.GetSpatialRef(),geom_type=ogr.wkbMultiPolygon )

In [62]:
# Add input Layer Fields to the output Layer if it is the one we want
inLayerDefn = inLayer.GetLayerDefn()
for i in range(0, inLayerDefn.GetFieldCount()):
    fieldDefn = inLayerDefn.GetFieldDefn(i)
    fieldName = fieldDefn.GetName()
#     if fieldName not in field_name_target:
#         continue
    outLayer.CreateField(fieldDefn)

In [63]:
# Get the output Layer's Feature Definition
outLayerDefn = outLayer.GetLayerDefn()

In [64]:
# Add features to the ouput Layer
for inFeature in inLayer:
    # Create output Feature
    outFeature = ogr.Feature(outLayerDefn)

    # Add field values from input Layer
    for i in range(0, outLayerDefn.GetFieldCount()):
        fieldDefn = outLayerDefn.GetFieldDefn(i)
        fieldName = fieldDefn.GetName()
#         if fieldName not in field_name_target:
#             continue

        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(),
            inFeature.GetField(i))
    
    # Set geometry as centroid
    geom = inFeature.GetGeometryRef()
    outFeature.SetGeometry(geom.Clone())
    # Add new feature to output Layer
    outLayer.CreateFeature(outFeature)
    outFeature = None

Warning 1: Value 68990516.4814999998 of field Shape_Area of feature 0 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 389968616.779999971 of field Shape_Area of feature 1 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 273113129.68599999 of field Shape_Area of feature 2 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 401922798.019999981 of field Shape_Area of feature 3 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 389968616.779999971 of field Shape_Area of feature 4 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 282297059.828000009 of field Shape_Area of feature 5 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 23678704.6403999999 of field Shape_A

Warning 1: Value 87416224.6423999965 of field Shape_Area of feature 68 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 153033874.522000015 of field Shape_Area of feature 69 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 127022616.623999998 of field Shape_Area of feature 70 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 170552420.645000011 of field Shape_Area of feature 71 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 85784072.0028000027 of field Shape_Area of feature 72 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 273863654.859000027 of field Shape_Area of feature 73 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 53811161.9407000095 of field 

Warning 1: Value 279938421.160000026 of field Shape_Area of feature 127 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 981141683.760999918 of field Shape_Area of feature 128 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 212523095.231999993 of field Shape_Area of feature 129 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 981141683.760999918 of field Shape_Area of feature 130 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 674901530.001999974 of field Shape_Area of feature 131 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 131902992.078999996 of field Shape_Area of feature 132 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 256231569.361000001 of 

Warning 1: Value 420220766.921999991 of field Shape_Area of feature 195 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 568540964.771999955 of field Shape_Area of feature 196 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 192974504.344000012 of field Shape_Area of feature 197 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 38586543.2678000033 of field Shape_Area of feature 198 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 38586543.2678000033 of field Shape_Area of feature 199 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 38586543.2678000033 of field Shape_Area of feature 200 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 847715912.843000054 of 

Warning 1: Value 348550747.124000013 of field Shape_Area of feature 257 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 233711007.100000024 of field Shape_Area of feature 258 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 309776997.588 of field Shape_Area of feature 259 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 339367257.728999972 of field Shape_Area of feature 260 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 187442046.578000009 of field Shape_Area of feature 261 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 342908508.29400003 of field Shape_Area of feature 262 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 27200414.3940000013 of field S

...

Warning 1: Value 206106558.612999976 of field Shape_Area of feature 673 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 10116595.0569000002 of field Shape_Area of feature 674 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 11158579.6347000003 of field Shape_Area of feature 675 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 22342619.8942000009 of field Shape_Area of feature 676 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 25128801.690200001 of field Shape_Area of feature 681 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 14528129.0427999999 of field Shape_Area of feature 682 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 10000528.1497000009 of f

Warning 1: Value 10222511.6612 of field Shape_Area of feature 766 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 9700080986.17000008 of field Shape_Area of feature 769 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 2666168892.38999987 of field Shape_Area of feature 770 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 70624878.6644999981 of field Shape_Area of feature 777 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 2277951506.67000008 of field Shape_Area of feature 778 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 3010432102.76000023 of field Shape_Area of feature 779 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 10183427.5055 of field Shape_

Warning 1: Value 42564698.1718000025 of field Shape_Area of feature 907 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 610539715.692000031 of field Shape_Area of feature 908 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 276321778.166000009 of field Shape_Area of feature 909 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 378938965.597000003 of field Shape_Area of feature 910 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 194348955.407999992 of field Shape_Area of feature 912 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 36446723.9306000024 of field Shape_Area of feature 914 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 34051590.7990000024 of 

Warning 1: Value 12274364.3628000002 of field Shape_Area of feature 999 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 41285579.8910999969 of field Shape_Area of feature 1003 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 110663648.358999997 of field Shape_Area of feature 1004 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 193166837.599999994 of field Shape_Area of feature 1005 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 21071278.5733999982 of field Shape_Area of feature 1007 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 11022099.1970000006 of field Shape_Area of feature 1008 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 22933361.967399999

Warning 1: Value 433933066.748000026 of field Shape_Area of feature 1128 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 3058834639.43999958 of field Shape_Area of feature 1129 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 24399494.8528999984 of field Shape_Area of feature 1130 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 24399494.8528999984 of field Shape_Area of feature 1131 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 11549978.6991000008 of field Shape_Area of feature 1136 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 12601920.1769999992 of field Shape_Area of feature 1147 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 4268046333.429999

In [65]:
# Save and close DataSources
inDataSource = None
outDataSource = None

<b>SY 2015-16</b>

In [66]:
inShapefile = '/Users/lacm/Documents/Stanford/*Capstone Research/Data/SY 2015-16/SABS/Clean/SABS_1516_Primary_Fixed_Geometries_Reprojected.shp'
inDriver = ogr.GetDriverByName("ESRI Shapefile")
inDataSource = inDriver.Open(inShapefile, 0)
inLayer = inDataSource.GetLayer()
# change filter for each year
inLayer.SetAttributeFilter(filter_2016)

0

In [67]:
# number of features after filter
inLayer.GetFeatureCount()

41795

In [68]:
# Create the output LayerS
outShapefile = os.path.join( os.path.split( inShapefile )[0], "3rd Grade Only SABS 2015-2016.shp" )
outDriver = ogr.GetDriverByName("ESRI Shapefile")

In [69]:
# Create the output shapefile
outDataSource = outDriver.CreateDataSource(outShapefile)
out_lyr_name = os.path.splitext( os.path.split( outShapefile )[1] )[0]
outLayer = outDataSource.CreateLayer( out_lyr_name, srs=inLayer.GetSpatialRef(),geom_type=ogr.wkbMultiPolygon )

In [70]:
# Add input Layer Fields to the output Layer if it is the one we want
inLayerDefn = inLayer.GetLayerDefn()
for i in range(0, inLayerDefn.GetFieldCount()):
    fieldDefn = inLayerDefn.GetFieldDefn(i)
    fieldName = fieldDefn.GetName()
#     if fieldName not in field_name_target:
#         continue
    outLayer.CreateField(fieldDefn)

In [71]:
# Get the output Layer's Feature Definition
outLayerDefn = outLayer.GetLayerDefn()

In [72]:
# Add features to the ouput Layer
for inFeature in inLayer:
    # Create output Feature
    outFeature = ogr.Feature(outLayerDefn)

    # Add field values from input Layer
    for i in range(0, outLayerDefn.GetFieldCount()):
        fieldDefn = outLayerDefn.GetFieldDefn(i)
        fieldName = fieldDefn.GetName()
#         if fieldName not in field_name_target:
#             continue

        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(),
            inFeature.GetField(i))
    
    # Set geometry as centroid
    geom = inFeature.GetGeometryRef()
    outFeature.SetGeometry(geom.Clone())
    # Add new feature to output Layer
    outLayer.CreateFeature(outFeature)
    outFeature = None

In [73]:
# Save and close DataSources
inDataSource = None
outDataSource = None